In [21]:
from openai import OpenAI
import pandas as pd
import numpy as np
import pandas as pd
import pickle
from transformers import AutoTokenizer, AutoModel
import torch
from tqdm import tqdm
tqdm.pandas()

In [80]:
client = OpenAI(
    base_url="https://api-inference.huggingface.co/v1/",
    api_key="secret"
)


def get_llm_answer(mes, client=client):
    messages = [
        {"role": "user", "content": mes}
    ]
    
    completion = client.chat.completions.create(
        model="Qwen/Qwen2.5-72B-Instruct", 
        messages=messages, 
        temperature=1
    )
    
    return completion.choices[0].message.content

In [81]:
def make_reflection(descr, author_solution, student_solution):
    promt = f"""
    Привет, ты помощник для решения задач студентам. Сейчас я тебе дам описание задачи, эталонный код (верный), код студента (неверный). Тебе нужно помочь найти ошибку. Но сначала, разбери этот код, сделай рефлексию, разбери каждую строчку кода и скажи, что делает код. Предположи, в чем может быть ошибка.

    Условие задачи:
    {descr}
    
    Эталонное решение автора:
    {author_solution}
    
    Решение студента:
    {student_solution}
    """
    ans = get_llm_answer(promt)
    return ans

In [82]:
def remake_answer(ans):
    prompt2 = f'''
Комментарий: {ans}

Изучи комментарий и определи тип ошибки:
1. Ошибка при использовании функций, переменных или операторов python
2. Ошибка, связанная с опечатками и забытыми скобками, двоеточиями, кавычками, f-строками и т.д
3. Ошибка, связанная с нерассмотрением всех случаев в ходе решения задачи, неверным выводом или логическая ошибка
4. Ошибка, связанная с обращением к неопределенной переменной
5. Ошибка табуляции

Далее переформулируй комментарий, чтобы ответ был стилистически похож на остальные из данной группы ошибок, опираясь на следующие условия:

Если речь об ошибке при использовании функций, переменных или операторов python:
1. Обратите внимание, что к словарям не применим метод .append().
2. Ошибка в определении переменной. Переменная info не задана.
3. Некорректно использован метод добавления элементов в список. Список не имеет метода .add().
4. Ошибка при использовании функции min(). Функция не может применяться к объекту класса str.
5. Ошибка при передаче аргумента функции range. Список не может счиаться аргументом функции, только переменная типа integer.
6. Ошибка при сравнении данных. Оператор "&" не используется для сравнения данных типа list.
7. Ошибка в открытых и скрытых тестах. \n\nОбратите внимание, что оператор "+" не применим при сравнении множеств.
8. Некорректно задана переменная. Словарь не имеет метода .append().
9. Обратите внимание, что после else не нужно прописывать никаких условий.
10. Ошибка в использовании метода проверки заглавных букв. Метод .upper() конвертирует все символы строки в заглавные.
11. Вы не добавили переменные, которые функция будет принимать как аргументы.
12. Ошибка при преобразовании переменной. int() не может применяться ко всему списку сразу. Примените int() к каждому элементу списка по отдельности.
13. Некорректно использована функция range(). Невозможно итерироваться на основе объекта класса str.

Если речь об ошибке, связанной с опечатками и забытыми скобками, двоеточиями, кавычками, f-строками и т.д:
1. Вы забыли поставить двоеточие после условия.
2. Вы забыли поставить закрывающую скобку функции print().
3. Вы забыли поставить двоеточие после условия в цикле while.
4. Вы забыли поставить знак нижнего подчеркивания в кавычки.
5. Вы забыли поставить скобки в методе .split().
6. Вы забыли поставить двоеточие после определения функции.
7. Вы забыли заключить значение Москва в кавычки.
8. Синтаксическая ошибка. Вы забыли заключить знак "#" в кавычки.
9. Вы забыли добавить условие для изменение значения переменной-счетчика.
10. Вы забыли поставить префикс f перед строкой, переданной переменной res.
11. Вы забыли поставить закрывающую кавычку в f-строке.
12. Вы забыли разделить элементы переменной res.
13. Вы забыли заключить "айдентика" в каквычки.
14. Обратите внимание на неверный синтаксис булевых значений.

Если речь об ошибке, связанной с нерассмотрением всех случаев в ходе решения задачи, неверным выводом или это логическая ошибка:
1. Обратите внимание на неверный оператор сравнения — необходимо проверить, что цвет не находится в списке cite_project.
2. Ваш код охватывает не все возможные случаи. Например, он не проверяет, присутствует ли цвет в списке logo_project. Попробуйте дополнить условие if, чтобы охватить все возможные случаи.
3. Ваш код некорректно проверяет условия задания. Например, он считает количество проектов, которые состоят только из заглавных букв и в которых есть знак подчеркивания. Попробуйте изменить условие if, чтобы скорректировать ошибку.
4. Ваш код использует некорректную переменную для подсчета количества проектов, соответствующих условиям задания.
5. Ваш код некорректно обращается к каждому элементу переменной logo. Попробуйте изменить условие if, чтобы скорректировать ошибку.
6. Ваш код некорректно проверяет условия задания. Например, он некорректно проверяет, начинаются лиэлементы со знака "#". Попробуйте изменить условие if, чтобы скорректировать ошибку.
7. Ваш код не проверяет все условия задания. Например, он некорректно выполняет условие "В конце программа печатает все хэштеги через запятую с пробелом". Дополните функцию print(), чтобы выполнить данное условие.
8. Ваш код выводит некорректную переменную.
9. Ваш код некорректно выполняет условия задания. Так, он не выполянет условие "не Москва или Санкт-Петербург". Попробуйте изменить условие if, чтобы скорректировать ошибку.
10. Ваш код не выполняет условия задания "возвращать фамилию дизайнера, который работает над самым легким проектом". Попробуйте изменить условие оператора return, чтобы скорректировать ошибку.
11. Ваш код некорректно выводит данные. Присутствуют дополнительные знаки переноса строки. Попробуйте изменить переменную line, чтобы скорректировать ошибку.
12. Ваш код некорректно проверяет условия задания. Например, он не проверяет есть ли в проектах знак подчеркивания. Попробуйте изменить условие if, чтобы скорректировать ошибку.
13. Ваш код использует неверный параметр в функции split().
14. Ваш код имеет некорректный формат вывода ответов. Помните, что если хотя бы одна глава была переведена, то функция должна возвращать список из двух значений: логической константы True и целого числа — количества слов в главах, которые были переведены.
15. Необходимо использовать одинаковые названия переменных.
16. В данном случае не нужно применять функцию int() к переменной color.
17. Обратите внимание на некорректное использование переменной для прерывания цикла while.
18. Ошибка в методе подсчета количества проектов, соответствующих условиям задания.
19. Некорректно использован метод добавления элементов в список. В список добавлена некорректная переменная.

Если речь об ошибке, связанной с обращением к неопределенной переменной:
1. Вы использовали переменную, которая не была задана ранее. Проверьте, что все переменные используются корректно.
2. Вы использовали переменную, которая не была задана ранее. Проверьте в условии цикла for, что все переменные заданы корректно.
3. Синтаксическая ошибка. Вы пытаетесь обратиться к переменной, которую не определили ранее. Убедитесь, что корректно написали все переменные.

Если это ошибка табуляции:
1. Обратите внимание на ошибку в табуляции условия if.
2. Обратите внимание на ошибку в табуляции выражения break.
3. Вы некорректно поставили отступы перед функцией print().

Комментарий не должен содержать код для решения задачи, он должен только натолкнуть на правильный ответ, а не содержать его.
В ответе я жду только переформулированный комментарий:
'''
    ans = get_llm_answer(prompt2)
    return ans

In [83]:
def get_anwer(descr, author_solution, student_solution):
    prompt1 = f'''
    Пример 1:
    Условие задачи:
    Анна и Галина выбирают, в каком агентстве заказать айдентику для бренда одежды. Реализуйте программу, которая поможет им сделать выбор:\n\nВ переменную anna_choice считывается список строк (названия агенств, которым доверяет Анна). Этот код уже написан.\nВ переменную galina_choice считывается список строк (названия агенств, которым доверяет Галина). Этот код уже написан.\nВ переменную top10 считывается список строк (названия агенств, входящих в топ-10).\nГарантируется, что для каждого списка будет введено хотя бы одно значение.\nПрограмма выводит через запятую с пробелом названия агентств из топ-10, которым одновременно доверяют и Анна, и Галина.\nНазвания должны быть отсортированы в алфавитном порядке.\nЕсли таких агенств нет, программа ничего не выводит.

    Эталонное решение автора:
    anna_choice = input().split()\ngalina_choice = input().split()\ntop10 = input().split()\n\nprint(', '.join(sorted((set(anna_choice) & set(galina_choice)).intersection(set(top10)))))

    Решение студента:
    anna_choice = input().split()\ngalina_choice = input().split()\ntop10 = input().split()\n\nprint(', '.join(sorted((set(anna_choice) + set(galina_choice)).intersection(set(top10)))))

    Подсказка:
    Ошибка в открытых и скрытых тестах. \n\nОбратите внимание, что оператор "+" не применим при сравнении множеств.

    Пример 2:
    Условие задачи:
    Реализуйте программу, которая напечатает хэштеги из фраз для лендинга сайта: \n\nВ переменную logo считывается текст лендинга. Этот код уже написан.\nПрограмма проверяет, есть ли в этом тексте хэштеги.  Хэштегами можно считать все элементы, которые начинаются со знака "#" и которые не состоят только из цифр (не включая знак "#"). Гарантируется, что хэштеги отделены от другого текста пробелом. \nВ конце программа печатает все хэштеги через запятую с пробелом. Хэштеги выводятся в том же порядке, в котором они идут в тексте.\nЕсли знаков хэштегов в тексте нет, то программа ничего не печатает.

    Эталонное решение автора:
    logo = input()\n\nres = []\nfor info in logo.split():\n    if info.startswith('#') and info[1:].isdigit() == False:      \n        res.append(info)\n        \nprint(*res, sep=', ')

    Решение студента:
    logo = input()\n\nres = []\nfor info in logo.split()\n    if info.startswith('#') and info[1:].isdigit() == False:      \n        res.append(info)\n        \nprint(*res, sep=', ')
    
    Подсказка:
    Вы забыли поставить двоеточие после условия в цикле for.

    Пример 3:
    Условие задачи:
    Лолита изучает, какие студии дизайна чаще всего дают комментарии в ее любимом журнале. Помогите определить студии, встретившиеся в тексте более двух раз. Гарантируется, что в тексте только названия сервисов написаны в кавычках «». \n\nФОРМАТ ВВОДА\nТекстовый файл designtalks.txt, в котором записан текст из журнала. \nГарантируется, что пустых строк в файле нет.\nГарантируется, что в файле будет упоминание хотя бы одной студии.\nТакже дана строка punctuation с символами, которые могут встречаться в тексте — необходимо будет очистить от них текст.\nФОРМАТ ВЫВОДА\nСтудии дизайна, встретившиеся в тексте более двух раз.\nСтудии должны быть записаны в обратном алфавитном порядке через запятую и пробел.\nДОПОЛНИТЕЛЬНО\nВ открытом тесте используется файл designtalks.txt. Вы можете скачать его и проверить свое решение локально.\nВ примере в первой колонке отображается номер теста (0 — открытый тест), считывать это число или использовать его в вашем решении не нужно.\nАЛГОРИТМ РЕШЕНИЯ \nЭто один из возможных алгоритмов решения задачи. Вы можете воспользоваться им в качестве подсказки или решить задачу любым другим удобным для вас способом.\nСоздайте пустой словарь, в который будете записывать слова и их количество.\nОткройте файл в режиме чтения в кодировке utf-8.\nОбратитесь к каждой строке в файле и выполните следующие действия:\nуберите невидимые символы в конце строки (символ перехода на новую строку);\nобратитесь к каждому символу в строке punctuation и используйте его для замены на пустую строку в строке файла;\nразбейте строку по разделителю, в результате получите список;\nобратитесь к каждому элементу в получившемся списке. Если первый символ — это кавычка «, тогда это студия дизайна, и нужно проверить:\nесли этого названия еще нет в словаре из п.1, то нужно его добавить в качестве ключа, присвоив значение 1.\nесли такое название уже есть в словаре из п.1, то нужно увеличить его значение на 1.\nПосле цикла создайте пустой список — в него нужно будет добавить те названия, значение которых в словаре более 2.\nОбратитесь к парам ключ-значение в словаре: если значение строго больше 2, то добавьте ключ в список из п.4.\nОтсортируйте список в обратном алфавитном порядке и напечатайте его элементы через запятую и пробел.

    Эталонное решение автора:
    punctuation = '!#$%&()*+,-./:;<=>?@^_'\n\ndata = {{}}\n\nwith open('designtalks.txt', 'r', encoding='utf-8') as my_file:\n    for line in my_file:\n        line = line.strip()\n        \n        for p in punctuation:\n            line = line.replace(p, '')\n        \n        line = line.split()\n        for w in line:\n            if w[0] == '«':\n                if w not in data:\n                    data[w] = 1\n                else:\n                    data[w] += 1\n                    \nres = []\n\nfor k, v in data.items():\n    if v > 2:\n        res.append(k)\n        \nprint(*sorted(res, reverse=True), sep=', ')

    Решение студента:
    punctuation = '!#$%&()*+,-./:;<=>?@^_'\n\ndata = {{}}\n\nwith open('designtalks.txt', 'r', encoding='utf-8') as my_file:\n    for line in my_file:\n        line = line.strip()\n        \n        for p in punctuation:\n            line = line.replace(p, '')\n        \n        line = line.split()\n        for w in line:\n            if w[0] == '«':\n                if w not in data:\n                    data[w] = 1\n                else:\n                    data[w] += 1\n                    \nres = []\n\nfor k, v in data.items():\n    if v > 2:\n        res.append(v)\n        \nprint(*sorted(res))

    Подсказка:
    Ошибка в открытых и скрытых тестах.

    \n\nВаш код некорректно выполняет условия задания. Например, он не выполняет условие "cтудии должны быть записаны в обратном алфавитном порядке через запятую и пробел". Попробуйте изменить условную конструкцию if, чтобы скорректировать ошибку.

    Пример 4:
    Условие задачи:
    Реализуйте программу, определяющую слова, которые нужно переводить:\n\nВ переменную N считывается целое число — количество записей, которые будут введены. Этот код уже написан.\nВ цикле while считывается N слов. \nГарантируется, что будет введена как минимум одна строка.\nНапечатайте количество слов, в которых буква "а" вне зависимости от регистра встречается не менее двух раз.

    Эталонное решение автора:
    N = int(input())\n\n# ваш код ниже\n\nresult = 0\n\nwhile N != 0:\n    info = input()\n    \n    if  info.lower().count('а') >= 2:\n        result += 1\n    \n    N -= 1\n    \nprint(result)

    Решение студента:
    N = int(input())\n\n# ваш код ниже\n\nresult = 0\n\nwhile N != 0:\n    info = input()\n    \n    if  inf.lower().count('а') >= 2:\n        result += 1\n    \n    N -= 1\n    \nprint(result)
    
    Подсказка:
    Синтаксическая ошибка. Вы пытаетесь обратиться к переменной, которую не определили ранее. Убедитесь, что корректно написали все переменные.

    Пример 5:
    Условие задачи:
    Реализуйте программу, которая проверит, что цвет используется только в проекте по созданию логотипа, но не в проекте по созданию дизайна сайта:\n\nДаны два списка logo_project и cite_project с кодами используемых цветов (строки).\nВ переменную color считывается код цвета (строка). Этот код уже написан.\nПрограмма должна проверять, что код цвета color есть только в списке logo_project, и если да, то печатать True. \nВ остальных случаях программа печатает False.

    Эталонное решение автора:
    logo_project = ['#a7a8f0', '#a7f0ca', '#b3b4e4', '#e4b3cd', '#e4e3b3', '#c0ced7']\ncite_project = ['#e4e3b3', '#a7a8f0', '#ccb1e6', '#b4f99e', '#f9b59e', '#c0ced7']\n\ncolor = input()\n\nif color in logo_project and not(color in cite_project):\n    print(True)\nelse:\n    print(False)

    Решение студента:
    logo_project = ['#a7a8f0', '#a7f0ca', '#b3b4e4', '#e4b3cd', '#e4e3b3', '#c0ced7']\ncite_project = ['#e4e3b3', '#a7a8f0', '#ccb1e6', '#b4f99e', '#f9b59e', '#c0ced7']\n\ncolor = input()\n\nif color in logo_project and not(color in cite_project):\n    print(True)\nelse:\nprint(False)
    
    Подсказка:
    Вы некорректно поставили отступы перед функцией print().

    Теперь ваша очередь:
    Условие задачи:
    {descr}

    Эталонное решение автора:
    {author_solution}

    Решение студента:
    {student_solution}
    

    На основе данной информации:
    1. Проанализируйте решение студента (без комментариев).
    2. Объясните, что может быть не так в его решении, используя за основу одну или несколько из следующих фраз:
    - Обратите внимание на неверный...
    - ...необходимо проверить, что...
    - Вы забыли поставить...
    - Необходимо использовать...
    - Вы некорректно...
    - Проверьте написание...
    - В данном случае не нужно...
    - ...неверный синтаксис...
    - Ваш код охватывает не все возможные случаи...
    - Попробуйте дополнить...
    - Вы использовали неверную...
    - Ошибка при обращении к...
    - Проверьте, что...
    - Ваш код использует неверный...

    Важные уточнения:
    Полное решение (код) предоставлять запрещено, тебе надо помочь студенту найти ошибку.

    Верных решений, к которым комментариев не требуется, не будет. Каждое решение содержит ошибку, которую тебе надо найти.
    
    Дайте полезную и краткую подсказку (не более трёх предложений), которая явно укажет на ошибку в решении, не предоставляя полного решения задачи.
    '''
    answer = get_llm_answer(prompt1)
    return answer

In [84]:
descr = """Реализуйте программу, которая проверит, что цвет используется только в проекте по созданию логотипа, но не в проекте по созданию дизайна сайта:

Даны два списка logo_project и cite_project с кодами используемых цветов (строки).
В переменную color считывается код цвета (строка). Этот код уже написан.
Программа должна проверять, что код цвета color есть только в списке logo_project, и если да, то печатать True. 
В остальных случаях программа печатает False. 
"""
author_solution = """logo_project = ['#a7a8f0', '#a7f0ca', '#b3b4e4', '#e4b3cd', '#e4e3b3', '#c0ced7']
cite_project = ['#e4e3b3', '#a7a8f0', '#ccb1e6', '#b4f99e', '#f9b59e', '#c0ced7']

color = input()

if color in logo_project and not(color in cite_project):
    print(True)
else:
    print(False)
"""
student_solution = """logo_project = ['#a7a8f0', '#a7f0ca', '#b3b4e4', '#e4b3cd', '#e4e3b3', '#c0ced7']
cite_project = ['#e4e3b3', '#a7a8f0', '#ccb1e6', '#b4f99e', '#f9b59e', '#c0ced7']

color = input()

if color in logo_project and color in cite_project:
    print(True)
else:
    print(False)
"""
ans = get_anwer(descr, author_solution, student_solution)
ans

'Обратите внимание на неверный логический оператор в условии if. Необходимо проверить, что цвет присутствует только в списке logo_project и отсутствует в списке cite_project. Проверьте, что в условии используется правильный логический оператор для проверки отсутствия цвета во втором списке.'

In [86]:
model_name = 'DeepPavlov/rubert-base-cased-sentence'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

In [87]:
def get_sentence_embedding(sentence):
    inputs = tokenizer(sentence, return_tensors='pt', truncation=True, padding=True, max_length=256)
    with torch.no_grad():
        outputs = model(**inputs)
        embedding = outputs.last_hidden_state[:, 0, :].squeeze().tolist()
    embedding_str = ' '.join(map(str, embedding))
    return embedding_str

In [89]:
test = pd.read_csv("hse/hse-test.csv", index_col=0)
test

,id_solution,student_solution,author_comment,author_comment_embedding,description,author_solution
0,0,discount = float(input())\nmoney = int(input(...,Ваше предсказание,NaN,"Реализуйте программу, которая напечатает стоим...",discount = float(input())\nmoney = int(input(...
1,1,discount = float(input())\nmoney = int(input(...,Ваше предсказание,NaN,"Реализуйте программу, которая напечатает стоим...",discount = float(input())\nmoney = int(input(...
2,2,discount = float(input())\nmoney = int(input(...,Ваше предсказание,NaN,"Реализуйте программу, которая напечатает стоим...",discount = float(input())\nmoney = int(input(...
3,3,discount = float(input())\nmoney = int(input(...,Ваше предсказание,NaN,"Реализуйте программу, которая напечатает стоим...",discount = float(input())\nmoney = int(input(...
4,4,discount = float(input())\nmoney = int(input(...,Ваше предсказание,NaN,"Реализуйте программу, которая напечатает стоим...",discount = float(input())\nmoney = int(input(...
...,...,...,...,...,...,...
320,725,def success(data):\n \n wins = 0\n mo...,Ваше предсказание,NaN,"Напишите функцию, определяющую успешна студия ...",def success(data):\n \n wins = 0\n mo...
321,726,def success(data):\n \n wins = 0\n mo...,Ваше предсказание,NaN,"Напишите функцию, определяющую успешна студия ...",def success(data):\n \n wins = 0\n mo...
322,727,def success(data):\n \n wins = 0\n mo...,Ваше предсказание,NaN,"Напишите функцию, определяющую успешна студия ...",def success(data):\n \n wins = 0\n mo...
323,728,def success(data):\n \n wins = 0\n mo...,Ваше предсказание,NaN,"Напишите функцию, определяющую успешна студия ...",def success(data):\n \n wins = 0\n mo...


In [ ]:
import re

def remove_comments(code):
    single_line_comment = r'(?m)^\s*#.*$'
    
    multi_line_comment = r'(?s)/\*.*?\*/'
    code = re.sub(single_line_comment, '', code) 
    code = re.sub(multi_line_comment, '', code)
    
    return code

test['student_solution'] = test['student_solution'].apply(remove_comments)

In [76]:
for i in tqdm(range(0, len(test))):
    descr = test.loc[i, "description"]
    student_solution = test.loc[i, "student_solution"]
    author_solution = test.loc[i, "author_solution"]
    last_ans = get_anwer(descr, author_solution, student_solution)
    ans = remake_answer(last_ans)
    print(i)
    print(last_ans)
    print("-------------------")
    print(ans)
    
    test.loc[i, "author_comment"] = ans
    test.to_csv("hse/test_remake_ans3.csv")

  0%|▌                                                                                                                                                                                                        | 1/325 [00:05<27:50,  5.16s/it]

0
Обратите внимание на неверный синтаксис f-строки. В вашем решении переменные внутри строки не обрабатываются как переменные, необходимо использовать фигурные скобки `{}` для их включения.
-------------------
Вы забыли поставить фигурные скобки `{}` вокруг переменных в f-строке.


  1%|█▏                                                                                                                                                                                                       | 2/325 [00:09<24:19,  4.52s/it]

1
Обратите внимание на неверный синтаксис в строке с принтом: вы забыли закрыть скобку в конце строки.
-------------------
Вы забыли поставить закрывающую скобку функции print().


  1%|█▊                                                                                                                                                                                                       | 3/325 [00:14<26:46,  4.99s/it]

...

322
Обратите внимание на неверный возврат значения в функции. В条件下，当工作室在所有年份都赢得了奖项时，函数应该返回 `True`，而当至少有一个年份没有赢得奖项时，应该返回累积的净利润。学生在函数的返回值中加入了 `False`，这是多余的。应该直接返回 `money`，而不是 `[False, money]`。

请注意，我尽量按照你的要求保持了中文和英文的混合使用，
-------------------
Ваш код некорректно выполняет условия задания. Например, когда студия выигрывает в所有年份, функция должна возвращать `True`, а если хотя бы в один год студия не выигрывает, то функция должна возвращать накопленную прибыль. Попробуйте изменить условие оператора return, чтобы скорректировать ошибку. В данном случае, возврат значения `[False


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▍| 324/325 [36:52<00:09,  9.10s/it]

323
Обратите внимание на неверный возврат значения в конце функции. Вместо возврата суммы чистой прибыли, вы возвращаете количество побед. Необходимо использовать метод `return money` для возврата суммы чистой прибыли, если студия не выигрывала премию во все годы.
-------------------
Ваш код некорректно выполняет условия задания. Вместо возврата суммы чистой прибыли, вы возвращаете количество побед. Необходимо использовать метод `return money` для возврата суммы чистой прибыли, если студия не выигрывала премию во все годы. Попробуйте изменить условие оператора return, чтобы скорректировать ошибку.


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 325/325 [36:58<00:00,  6.82s/it]

324
Обратите внимание на неверный возврат значений из функции. Вы используете `print` вместо `return` для возврата результата. Функция должна возвращать значения с помощью `return`.
-------------------
Обратите внимание, что функция должна возвращать значения с помощью `return`, а не выводить их с помощью `print`.


In [77]:
test

,id_solution,student_solution,author_comment,author_comment_embedding,description,author_solution
0,0,discount = float(input())\nmoney = int(input(...,Вы забыли поставить фигурные скобки `{}` вокру...,NaN,"Реализуйте программу, которая напечатает стоим...",discount = float(input())\nmoney = int(input(...
1,1,discount = float(input())\nmoney = int(input(...,Вы забыли поставить закрывающую скобку функции...,NaN,"Реализуйте программу, которая напечатает стоим...",discount = float(input())\nmoney = int(input(...
2,2,discount = float(input())\nmoney = int(input(...,Вы забыли поставить закрывающую фигурную скобк...,NaN,"Реализуйте программу, которая напечатает стоим...",discount = float(input())\nmoney = int(input(...
3,3,discount = float(input())\nmoney = int(input(...,Вы забыли правильно расставить переменные в f-...,NaN,"Реализуйте программу, которая напечатает стоим...",discount = float(input())\nmoney = int(input(...
4,4,discount = float(input())\nmoney = int(input(...,Ваш код некорректно выполняет условия задания....,NaN,"Реализуйте программу, которая напечатает стоим...",discount = float(input())\nmoney = int(input(...
...,...,...,...,...,...,...
320,725,def success(data):\n \n wins = 0\n mo...,Ваш код некорректно выполняет условия задания....,NaN,"Напишите функцию, определяющую успешна студия ...",def success(data):\n \n wins = 0\n mo...
321,726,def success(data):\n \n wins = 0\n mo...,Ваш код некорректно выполняет условия задания....,NaN,"Напишите функцию, определяющую успешна студия ...",def success(data):\n \n wins = 0\n mo...
322,727,def success(data):\n \n wins = 0\n mo...,Ваш код некорректно выполняет условия задания....,NaN,"Напишите функцию, определяющую успешна студия ...",def success(data):\n \n wins = 0\n mo...
323,728,def success(data):\n \n wins = 0\n mo...,Ваш код некорректно выполняет условия задания....,NaN,"Напишите функцию, определяющую успешна студия ...",def success(data):\n \n wins = 0\n mo...


In [78]:
test['author_comment_embedding'] = test['author_comment'].apply(lambda x: get_sentence_embedding(x))
test

,id_solution,student_solution,author_comment,author_comment_embedding,description,author_solution
0,0,discount = float(input())\nmoney = int(input(...,Вы забыли поставить фигурные скобки `{}` вокру...,-0.2070768177509308 -1.5525593757629395 -0.181...,"Реализуйте программу, которая напечатает стоим...",discount = float(input())\nmoney = int(input(...
1,1,discount = float(input())\nmoney = int(input(...,Вы забыли поставить закрывающую скобку функции...,-0.19086310267448425 -1.088375449180603 -0.212...,"Реализуйте программу, которая напечатает стоим...",discount = float(input())\nmoney = int(input(...
2,2,discount = float(input())\nmoney = int(input(...,Вы забыли поставить закрывающую фигурную скобк...,-0.22407527267932892 -1.3472121953964233 -0.01...,"Реализуйте программу, которая напечатает стоим...",discount = float(input())\nmoney = int(input(...
3,3,discount = float(input())\nmoney = int(input(...,Вы забыли правильно расставить переменные в f-...,-0.3937441110610962 -1.8197307586669922 -0.082...,"Реализуйте программу, которая напечатает стоим...",discount = float(input())\nmoney = int(input(...
4,4,discount = float(input())\nmoney = int(input(...,Ваш код некорректно выполняет условия задания....,-0.396786630153656 -1.4490821361541748 0.21602...,"Реализуйте программу, которая напечатает стоим...",discount = float(input())\nmoney = int(input(...
...,...,...,...,...,...,...
320,725,def success(data):\n \n wins = 0\n mo...,Ваш код некорректно выполняет условия задания....,-0.27817776799201965 -1.557130217552185 0.1066...,"Напишите функцию, определяющую успешна студия ...",def success(data):\n \n wins = 0\n mo...
321,726,def success(data):\n \n wins = 0\n mo...,Ваш код некорректно выполняет условия задания....,-0.38985639810562134 -1.5489704608917236 0.054...,"Напишите функцию, определяющую успешна студия ...",def success(data):\n \n wins = 0\n mo...
322,727,def success(data):\n \n wins = 0\n mo...,Ваш код некорректно выполняет условия задания....,-0.3556545078754425 -1.5638694763183594 0.0662...,"Напишите функцию, определяющую успешна студия ...",def success(data):\n \n wins = 0\n mo...
323,728,def success(data):\n \n wins = 0\n mo...,Ваш код некорректно выполняет условия задания....,-0.26897576451301575 -1.4577138423919678 -0.02...,"Напишите функцию, определяющую успешна студия ...",def success(data):\n \n wins = 0\n mo...


In [79]:
test[["id_solution", "author_comment", "author_comment_embedding"]].rename({
    "id_solution": "solution_id"
}, axis=1).to_csv("hse/qwen_few_shot_reflection_remake_ans3.csv", index=False)